In [228]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point, Polygon
import shapely
import pickle

In [229]:
data = pd.read_csv('./Cal1.1.csv')
TSLF = pd.read_csv('./TSLF/TSLF.csv')
data.drop(columns = ['Unnamed: 0' , 'Event_ID' , 'Grass_Burned(ha)' , 'geometry'] , inplace = True)
TSLF.drop(columns = ['index' , 'Unnamed: 0'] , inplace = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1697 entries, 0 to 1696
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Fire_Name              1697 non-null   object 
 1   Lat                    1697 non-null   float64
 2   Long                   1697 non-null   float64
 3   Fire_Year              1697 non-null   object 
 4   Fire_Month             1697 non-null   int64  
 5   Total_area_burned(ha)  1697 non-null   float64
 6   Unchanged(ha)          1697 non-null   float64
 7   Low_Severity(ha)       1697 non-null   float64
 8   Moderate_Severity(ha)  1697 non-null   float64
 9   High_Severity(ha)      1697 non-null   float64
 10  FuelTreatment          1697 non-null   int64  
dtypes: float64(7), int64(2), object(2)
memory usage: 146.0+ KB


In [230]:
TSLF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1697 entries, 0 to 1696
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Fire_Name              1697 non-null   object 
 1   Lat                    1697 non-null   float64
 2   Long                   1697 non-null   float64
 3   Fire_Year              1697 non-null   int64  
 4   Fire_Month             1697 non-null   int64  
 5   Total_area_burned(ha)  1697 non-null   float64
 6   High_Severity(ha)      1697 non-null   float64
 7   TSLF                   1697 non-null   int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 106.2+ KB


In [231]:
data['Fire_Year'] = pd.to_datetime(data['Fire_Year'], format='%Y-%m-%d %H:%M:%S', utc=True)
data['Fire_Year'] = data['Fire_Year'].dt.year
data['Fire_Year'] = data['Fire_Year'].apply(pd.to_numeric)
data.head()


,Fire_Name,Lat,Long,Fire_Year,Fire_Month,Total_area_burned(ha),Unchanged(ha),Low_Severity(ha),Moderate_Severity(ha),High_Severity(ha),FuelTreatment
0,152,37.096,-121.167,2003,6,689.94,1.17,88.38,566.19,34.20,0
1,16,38.893,-122.265,2002,10,615.69,0.54,557.28,55.98,0.99,0
2,165,37.225,-120.834,2001,6,726.03,36.27,146.52,338.76,204.39,0
3,241 FIRE,33.828,-117.747,2007,3,750.33,17.46,515.16,217.17,0.54,0
4,36,40.300,-122.734,2005,8,917.55,0.00,619.29,295.92,1.71,0


In [232]:
TSLF.head()

,Fire_Name,Lat,Long,Fire_Year,Fire_Month,Total_area_burned(ha),High_Severity(ha),TSLF
0,152,37.096,-121.167,2003,6,689.94,34.20,0
1,16,38.893,-122.265,2002,10,615.69,0.99,1
2,165,37.225,-120.834,2001,6,726.03,204.39,1
3,241 FIRE,33.828,-117.747,2007,3,750.33,0.54,0
4,36,40.300,-122.734,2005,8,917.55,1.71,0


In [233]:
data.sort_values(by = ['Lat', 'Long'], ascending = True, inplace = True) #arbitrary sort order for TSLFmerge
TSLF.sort_values(by = ['Lat' , 'Long'], ascending = True, inplace = True) #arbitrary sort order for TSLFmerge

In [234]:
data['TSLF(months)'] = TSLF[['TSLF']]
data

,Fire_Name,Lat,Long,Fire_Year,Fire_Month,Total_area_burned(ha),Unchanged(ha),Low_Severity(ha),Moderate_Severity(ha),High_Severity(ha),FuelTreatment,TSLF(months)
157,BORDER 50,32.577,-116.722,2005,10,1239.57,0.18,466.47,674.64,98.10,0,0
158,BORDER 6,32.579,-116.264,2012,5,45.45,1.17,28.53,15.75,0.00,0,0
915,MINE,32.581,-116.768,1986,9,417.69,0.09,345.06,72.27,0.27,0,0
155,BORDER,32.585,-116.514,2004,9,363.24,0.00,56.07,304.02,3.15,0,0
536,GLORIA,32.589,-116.443,1999,10,213.66,0.00,76.23,136.44,0.99,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
480,FLETCHER,41.971,-120.545,2007,7,2680.92,7.47,692.10,998.91,979.74,0,1
817,LONGWOOD,41.997,-123.634,1987,8,2300.13,0.00,579.42,834.21,886.50,0,1
87,BARRY POINT,42.015,-120.681,2012,8,15471.00,32.49,6599.97,5690.97,3059.10,0,1
107,BEAVER COMPLEX,42.029,-122.250,2014,7,3778.20,11.43,2542.32,1124.91,99.27,0,9


In [235]:
data.to_csv('Cal1.2.csv')